In [ ]:
select count(*) from "tsql-dml".dbo.Posts

In [ ]:
--WHERE CLAUSE refresher and operators

select * from Posts
WHERE 
    score > 30
    AND Tags like '%machine-learning%'
    AND YEAR(CreationDate) IN (2014,2015,2018)

    --carefully see how we extracted the comments from comments table for filtered criteria defined above using Exists
    -- Exists() check for the existence of rows
    AND Exists(select id from Comments where PostId = Posts.Id)

    AND FavoriteCount BETWEEN 40 AND 100

In [ ]:
-- insert statement variations

-- Specify all column names and their  corresponding values
-- values for identity column is not passed because it is aut0-incremented and SQL server automatically increments its value with every insert command


-- SET IDENTITY_INSERT USERS ON (run it, if you want to insert identity column value on your own).By default it is OFF
INSERT INTO [dbo].[Users] 
        ([Reputation]
        , [CreationDate]
        , [DisplayName]
        , [LastAccessDate]
        , [WebsiteUrl]
        , [Location]
        , [AboutMe]
        , [Views]
        , [UpVotes]
        , [DownVotes]
        , [ProfileImageUrl]
        , [AccountId]) 
    VALUES 
        (1
        , CAST(N'2016-06-22T17:35:14.680' AS DateTime)
        , N'xavier-morera'
        , CAST(N'2016-06-22T17:35:14.680' AS DateTime)
        , N'www.xaviermorera.com'
        , N'Costa Rica'
        , NULL
        , 0
        , 0
        , 0
        , N'http://www.xaviermorera.com/wp-content/uploads/2016/06/xavier-morera-150x150.jpg'
        , 17056189)

In [ ]:
-- provide values for only columns whose value cannot be null & NULL or default value (if set) is automatically set for columns whose value can be null
-- In short it is not required to pass values for column which can hold NULL values

-- inserting values only for those columns which cannot hold NULL values

INSERT INTO [dbo].[Users] 
        ([Reputation]
        , [CreationDate]
        , [DisplayName]
        , [LastAccessDate]
        , [UpVotes]
        , [DownVotes]
        , [AccountId]) 
    VALUES 
        (1
        , CAST(N'2016-06-22T17:35:14.680' AS DateTime)
        , N'steve-towers'
        , CAST(N'2016-06-22T17:35:14.680' AS DateTime)
        , 0
        , 0
        , 17056192)



In [ ]:
-- Bulk Insert

BULK INSERT Comments from  '/Users/mac/Desktop/transact-sql/comments.csv' 
WITH (
    FIRSTROW = 2,
    FORMAT= 'CSV'

)

In [ ]:
--Update statement
select * from Users where DisplayName='Xavier'

-- UPDATE Users SET    
--     UpVotes = 10,
--     DownVotes = 5
-- where DisplayName='Xavier'




In [ ]:
--@@Rowcount is used to obtain the number of rows affected by the update statement

declare @rowsAffected int;

 update Users set Views=1001 where DisplayName like '%Xavier%'

set @rowsAffected = @@Rowcount

IF @rowsAffected > 0 
PRINT CONVERT(VARCHAR,@rowsAffected)+ ' rows are affected after executing update statement'

select Id, displayname, views from Users where DisplayName like '%Xavier%'


In [ ]:
-- updating value based on the data in another table
update Posts 
set posts.Score += Comments.Score *  10
from Posts INNER JOIN Comments on (Posts.Id = Comments.PostId)
where Posts.PostTypeId=1

In [ ]:
--Deleting

--based on condition in another table
delete FROM Comments
FROM COMMENTS JOIN POSTS ON Comments.PostId = Posts.Id
WHERE Posts.Score > 0

In [ ]:
--Deleting
--cascade delete

--drop table PostsCopy
CREATE TABLE PostsCopy (
		Id INT NOT NULL PRIMARY KEY IDENTITY(1,1),
		CreationDate DATETIME NOT NULL DEFAULT GETDATE(),
		Score INT NOT NULL DEFAULT 0,
		ViewCount INT,
		Title VARCHAR(500),
		Tags VARCHAR(255)
	);

In [ ]:
--Deleting
--cascade delete

-- drop table CommentsCopy
	CREATE TABLE CommentsCopy (
		Id INT NOT NULL PRIMARY KEY IDENTITY(1,1),
		UserId INT,
		PostId INT NOT NULL,
		Score INT NOT NULL DEFAULT 0,
		Text VARCHAR(MAX) NOT NULL,
		CreationDate DATETIME NOT NULL DEFAULT GETDATE(),
        -- defined the constraint with delete cascade
		CONSTRAINT FK_POSTID_COMMENTSCOPY FOREIGN KEY(PostId) REFERENCES PostsCopy (Id) ON DELETE CASCADE
	);

In [ ]:
--Deleting
--cascade delete

SET IDENTITY_INSERT PostsCopy ON;  
GO

-- copying the data from actual tables into demo tables. Notice the select instead of values clause that we typically use to write while inserting
INSERT PostsCopy (Id,CreationDate,Score,ViewCount,Title,Tags)
SELECT Id,CreationDate,Score,ViewCount,Title,Tags FROM Posts;

SET IDENTITY_INSERT PostsCopy OFF;  
GO

INSERT CommentsCopy (UserId, PostId, Score, Text, CreationDate)
SELECT UserId, PostId, Score, Text, CreationDate FROM Comments;


In [ ]:
--Deleting
--cascade delete

--Get the Post with the highest number of Comments
SELECT TOP(1) PostId, COUNT(PostId) AS [Number of Comments]
FROM [tsql-dml].[dbo].[Comments]
GROUP BY PostId
ORDER BY COUNT(PostId) DESC

SELECT * FROM CommentsCopy WHERE PostId = 30430;
select * from PostsCopy WHERE Id = 30430;


In [ ]:
--Deleting
--cascade delete

DELETE PostsCopy WHERE PostsCopy.Id = 30430;

-- after executing this go and run above statement you wont  find any results in CommentsCopy table 

In [ ]:
-- Truncate removes the data from table but the table structure and all constraints are not affected and stay same

In [ ]:
--maintaining data integrity

--select the id with max no of posts
--select OwnerUserId, COUNT(OwnerUserId) as TotalPosts from Posts group by OwnerUserId ORDER BY TotalPosts DESC

-- we cannot read the data from a transaction that is currently opened. important for consistency
-- OPENING transaction for a long time can affect other queries 

BEGIN TRANSACTION update_records;  --starting transaction

update Posts set 
    OwnerUserId = 75298,
    OwnerDisplayName= 'xavier-morera'

where OwnerUserId=836

update Comments set 
    UserId = 75298,
    UserDisplayName= 'xavier-morera'

where UserId=836

COMMIT TRANSACTION; -- ending transaction

In [ ]:
-- QUERYING DATA LOCKED IN TRANSACTION
-- USE WITH (NOLOCK) which retrieves the records regardless of locks



SELECT * FROM Posts 

WITH(NOLOCK) --this can be probalamitic and cause DIRTY READ. but it is faster. 
            --NOLOCK is same as READUNCOMMITTED 

WHERE OwnerUserId=75298

In [ ]:
-- Rollback

BEGIN TRANSACTION insert_another_user

INSERT INTO Users (DisplayName, Location, AboutMe)  VALUES ('another-tsql-user', 'San Jose, Costa Rica', 'This user will never exist!');

select * from Users where DisplayName='another-tsql-user'

-- we have not committed this transaction yet so lets roll back this`

In [ ]:
-- Rollback continued
ROLLBACK TRAN insert_another_user; 

select * from Users where DisplayName='another-tsql-user'

In [ ]:
-- partially undoing transaction with save points

-- used to rollback a transaction upto a certain point instead of rolling back entire transaction.
-- save transaction [name]

BEGIN TRAN;

DELETE FROM Comments WHERE Comments.Text LIKE '%python%';

SAVE TRAN del_python;

DELETE FROM Comments WHERE Comments.Text LIKE '%java%';

SAVE TRAN del_java;

SELECT * FROM Comments WHERE Comments.Text LIKE '%python%' OR Comments.Text LIKE '%java%';

In [ ]:
-- partially undoing continued

ROLLBACK TRAN del_python;
SELECT * FROM Comments WHERE Comments.Text LIKE '%python%' OR Comments.Text LIKE '%java%';

COMMIT TRAN;


--Note:
-- distributed transaction is an operaton on two databases retaining ACID properties
-- managed by microsoft distibuted transaction coordinator (MS DTC)

In [ ]:
-- ADVANCE TSQL TECHNIQUES

-- using insert with multiple values
	INSERT INTO [dbo].[Comments]
			([PostId]
			,[Score]
			,[Text]
			,[CreationDate]
			,[UserDisplayName]
			,[UserId])
		VALUES
			(10,0,N'This is one comment',CAST(N'2019-07-22T17:35:14.680' AS DateTime),N'xavier-morera',24),
			(10,0,N'This is a second comment',CAST(N'2019-07-22T17:35:14.680' AS DateTime),N'xavier-morera',24),
			(10,0,N'And even a third',CAST(N'2019-07-22T17:35:14.680' AS DateTime),N'xavier-morera',24)

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued

-- using select with insert
CREATE TABLE Employees (
		Id INT NOT NULL PRIMARY KEY IDENTITY(1,1),
		Reputation INT NOT NULL DEFAULT 0,
		CreationDate DATETIME NOT NULL DEFAULT GETDATE(),
		DisplayName VARCHAR(255) NOT NULL,
		Salary INT,
		JobTitle VARCHAR(255),
		Boss INT,
		Department VARCHAR(255)
	);

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued

-- using a select with insert continued

SET IDENTITY_INSERT Employees ON

--Insert users with a reputation higher than 5000 into the new table employees: 
	INSERT INTO Employees (Id, Reputation, CreationDate, DisplayName, Salary)

        -- there is not salary column in users so passing scalar value 0 
		SELECT Id, Reputation, CreationDate, DisplayName, 0
		FROM Users
		WHERE Users.Reputation > 5000

SET IDENTITY_INSERT Employees OFF

SELECT * FROM Employees

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued

-- using multiple select in insert to populate values form other table

INSERT INTO Comments (CreationDate, PostId, Score, Text, UserDisplayName, UserId) 
	VALUES (
		GETDATE(),

        -- using select  in insert values clause
		(SELECT Id FROM Posts WHERE Title = 'Parallel and distributed computing'),

		0,
		'I am the last comment!',
		'xavier-morera',

        -- using select  in insert values clause
		(SELECT Id FROM Users WHERE DisplayName = 'xavier-morera')

		);
        
SELECT * FROM Comments WHERE UserDisplayName = 'xavier-morera' ORDER BY CreationDate DESC;

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued
-- if we want to know what auto increment id is assign by the database after running current insert command

INSERT INTO Posts (PostTypeId, Body, OwnerUserId, OwnerDisplayName, Title)

    --print the values inserted
	OUTPUT INSERTED.Id, INSERTED.PostTypeId, inserted.OwnerUserId, inserted.OwnerDisplayName, inserted.Body

	VALUES (1,'This is a Post!',75298,'xavier-morera','This is a Post Title!');

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued

--Merge statement used to synchronize tables
-- used in a scenario where there is a need to insert, update and delete operations on the target table on conditional basis

--source table
CREATE TABLE CommentsSource (
		Id INT NOT NULL PRIMARY KEY,
		PostId INT NOT NULL,
		Score INT NOT NULL DEFAULT 0,
		Text VARCHAR(MAX) NOT NULL,
		CreationDate DATETIME NOT NULL DEFAULT GETDATE(),
		UserDisplayName VARCHAR(255),
		UserId INT
	);
--target table
	CREATE TABLE CommentsTarget (
		Id INT NOT NULL PRIMARY KEY,
		PostId INT NOT NULL,
		Score INT NOT NULL DEFAULT 0,
		Text VARCHAR(MAX) NOT NULL,
		CreationDate DATETIME NOT NULL DEFAULT GETDATE(),
		UserDisplayName VARCHAR(255),
		UserId INT
	);


In [ ]:
-- ADVANCE TSQL TECHNIQUES continued

--taking all rows from comments table except where text is python and inserting into comments source
INSERT INTO CommentsSource (Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId)

    --specifying which rows to inserted using except set operator
    SELECT Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId
	    FROM Comments 
    EXCEPT 
	(SELECT Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId
		FROM Comments WHERE Text LIKE '%python%');

--taking all rows from taget table except where text is python and inserting into comments target
INSERT INTO CommentsTarget (Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId)

	SELECT Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId
		FROM Comments 
    EXCEPT 
	(SELECT Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId
		FROM Comments WHERE Text LIKE '%java%');

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued
--Merging

MERGE CommentsTarget USING CommentsSource ON (CommentsTarget.Text = CommentsSource.Text)

		WHEN NOT MATCHED BY TARGET THEN	
			INSERT (Id,PostId,Score,Text,CreationDate,UserDisplayName,UserId)
			VALUES (
				CommentsSource.Id,
				CommentsSource.PostId,
				CommentsSource.Score,
				CommentsSource.Text,
				CommentsSource.CreationDate,
				CommentsSource.UserDisplayName,
				CommentsSource.UserId
			)
		
		WHEN NOT MATCHED BY SOURCE THEN 		
			DELETE;

In [ ]:
-- ADVANCE TSQL TECHNIQUES continued

-- bulk insert using BCP utility that is used to copy data between SQL server instance and data file in user specified format

